# `MCCS Customer Survey Responses` Sentiment Analysis

In [ ]:
from typing import NamedTuple, List, Dict, Union, Tuple, Optional
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import spacy_universal_sentence_encoder
from langcodes import Language
import pickle
import pandas as pd
import numpy as np

from src.scripts.utils import construct_path_from_project_root, resolve_import_path_from_project_root, generate_curr_date_to_append_to_filename

In [2]:
# Establishing CONSTANT values to use across my Jupyter Notebook
class CONSTANTS(NamedTuple):
    NUM_K_FOLDS = 10
    RANDOM_STATE = 5934
    RANGE_ITERATION_CROSS_VALIDATE = range(1, NUM_K_FOLDS + 1)

# Generating `Date Timestamp` for Saved `ML`-Models

In [3]:
date_timestamp = generate_curr_date_to_append_to_filename()
date_timestamp

'2025_03_12'

# Data Preprocessing

In [4]:
excel_workbook = pd.read_excel(resolve_import_path_from_project_root("../data-lake/CustomerSurveyResponses.xlsx"), sheet_name=None)
for (sheet_name, df_sheet) in excel_workbook.items():
    print(f"Displaying Sheet: {sheet_name}")
    
    num_rows_to_preview_display = 10
    if sheet_name == "Metadata":
        num_rows_to_preview_display = 20
    
    print(df_sheet.head(n=num_rows_to_preview_display))
    print(df_sheet.info())
    
    if "answerFreeTextValues" in set(df_sheet.columns):
        print(df_sheet["answerFreeTextValues"].notna())
        print(df_sheet["answerFreeTextValues"].notna().value_counts())
    
    print("\n\n\n")

Displaying Sheet: Metadata
      Customer Survey Responses from 1/2/25 - 2/17/25  \
0                                                 NaN   
1   The following sheets contain recent survey  fe...   
2                                                 NaN   
3                                              Column   
4                                        respondentId   
5                                          questionId   
6                                         reponseTime   
7                                        questionName   
8                                     questionaPhrase   
9                                        questionType   
10                                      questionLabel   
11                                       answerLabels   
12                                answerDisplayLabels   
13                                        answerTexts   
14                                       answerValues   
15                               answerFreeTextValues   
16  

In [5]:
expected_metadata_columns = list(excel_workbook["MainStores"].columns)
df_customer_survey_responses = pd.DataFrame(columns=["sheetName"] + expected_metadata_columns + ["sentiment"])

for (sheet_name, df_sheet) in excel_workbook.items():
    print(f"Processing Sheet: {sheet_name}")

    if sheet_name == "Metadata":
        continue  

    df_sheet = df_sheet.reindex(columns=expected_metadata_columns)
    df_sheet.dropna(subset=["answerFreeTextValues"], inplace=True)
    
    df_sheet["sheetName"] = sheet_name
    df_sheet["sentiment"] = "" # Will fill in later (manually)
    
    df_customer_survey_responses = pd.concat([df_customer_survey_responses, df_sheet], ignore_index=True)

display(df_customer_survey_responses.head())

Processing Sheet: Metadata
Processing Sheet: MainStores
Processing Sheet: MarineMarts
Processing Sheet: HospitalityServices
Processing Sheet: FoodBeverage


/var/folders/jx/_2p5x1js5y5_rcb0l3jwzhvm0000gn/T/ipykernel_1056/779681427.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_customer_survey_responses = pd.concat([df_customer_survey_responses, df_sheet], ignore_index=True)


,sheetName,respondentId,questionId,responseTime,questionName,questionPhrase,questionType,questionLabel,answerLabels,answerDisplayLabels,answerTexts,answerValues,answerFreeTextValues,sentiment
0,MainStores,NoEU4w4V910B5cBUo8VolA4C,SBD9516Q037,2025-02-17 15:24:00,What else would you like to share with us to h...,What else would you like to share with us to h...,CQ,OE_Improve,NaN,NaN,NaN,NaN,I was double charged for an item even said I t...,
1,MainStores,NoEU4w4V910B5cBUo8VolA4C,SBD9516Q039,2025-02-17 15:24:00,Please provide a brief description of your iss...,Please provide a brief description of your iss...,CQ,OE_Issue Description,NaN,NaN,NaN,NaN,I would like to be refunded the $10.99 plus ta...,
2,MainStores,NoEU4w4V910B5cBUo8VolA4C,CPPAUTOBd27631241,2025-02-17 15:24:00,storeid,storeid,CPP,storeid,NaN,NaN,NaN,NaN,15100,
3,MainStores,c1tZgAEEwJ5UkgZscBk8pA4C,CPPAUTOBd27631241,2025-02-17 13:54:01,storeid,storeid,CPP,storeid,NaN,NaN,NaN,NaN,1100,
4,MainStores,QlVRdM5kMp1Zg8dYhRpYlg4C,CPPAUTOBd27631241,2025-02-17 13:31:18,storeid,storeid,CPP,storeid,NaN,NaN,NaN,NaN,13100,


In [6]:
print(df_customer_survey_responses.info())
df_customer_survey_responses["answerFreeTextValues"].notna().value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3660 entries, 0 to 3659
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   sheetName             3660 non-null   object        
 1   respondentId          3660 non-null   object        
 2   questionId            3660 non-null   object        
 3   responseTime          3660 non-null   datetime64[ns]
 4   questionName          3660 non-null   object        
 5   questionPhrase        3660 non-null   object        
 6   questionType          3660 non-null   object        
 7   questionLabel         3660 non-null   object        
 8   answerLabels          0 non-null      object        
 9   answerDisplayLabels   0 non-null      object        
 10  answerTexts           0 non-null      object        
 11  answerValues          0 non-null      object        
 12  answerFreeTextValues  3660 non-null   object        
 13  sentiment         

answerFreeTextValues
True    3660
Name: count, dtype: int64

In [7]:
df_customer_survey_responses_filtered = df_customer_survey_responses.dropna(axis=1)
print(f"Columns Left: {set(df_customer_survey_responses_filtered.columns)}")
print(df_customer_survey_responses_filtered.info())
print(df_customer_survey_responses_filtered["answerFreeTextValues"].notna().value_counts())
df_customer_survey_responses_filtered

Columns Left: {'questionId', 'sheetName', 'questionPhrase', 'answerFreeTextValues', 'responseTime', 'questionLabel', 'respondentId', 'sentiment', 'questionType', 'questionName'}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3660 entries, 0 to 3659
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   sheetName             3660 non-null   object        
 1   respondentId          3660 non-null   object        
 2   questionId            3660 non-null   object        
 3   responseTime          3660 non-null   datetime64[ns]
 4   questionName          3660 non-null   object        
 5   questionPhrase        3660 non-null   object        
 6   questionType          3660 non-null   object        
 7   questionLabel         3660 non-null   object        
 8   answerFreeTextValues  3660 non-null   object        
 9   sentiment             3660 non-null   object        
dtypes: datetime64[

,sheetName,respondentId,questionId,responseTime,questionName,questionPhrase,questionType,questionLabel,answerFreeTextValues,sentiment
0,MainStores,NoEU4w4V910B5cBUo8VolA4C,SBD9516Q037,2025-02-17 15:24:00,What else would you like to share with us to h...,What else would you like to share with us to h...,CQ,OE_Improve,I was double charged for an item even said I t...,
1,MainStores,NoEU4w4V910B5cBUo8VolA4C,SBD9516Q039,2025-02-17 15:24:00,Please provide a brief description of your iss...,Please provide a brief description of your iss...,CQ,OE_Issue Description,I would like to be refunded the $10.99 plus ta...,
2,MainStores,NoEU4w4V910B5cBUo8VolA4C,CPPAUTOBd27631241,2025-02-17 15:24:00,storeid,storeid,CPP,storeid,15100,
3,MainStores,c1tZgAEEwJ5UkgZscBk8pA4C,CPPAUTOBd27631241,2025-02-17 13:54:01,storeid,storeid,CPP,storeid,1100,
4,MainStores,QlVRdM5kMp1Zg8dYhRpYlg4C,CPPAUTOBd27631241,2025-02-17 13:31:18,storeid,storeid,CPP,storeid,13100,
...,...,...,...,...,...,...,...,...,...,...
3655,FoodBeverage,GCCff7oOUUXiauSq4blkfl4a9EEuhfS7,CPPAUTOBd27492941,2025-01-03 07:14:35,restaurantName,restaurantName,CPP,restaurantName,Hangar One,
3656,FoodBeverage,GCCff7oOUUXiauSq4blkfl4a9EEuhfS7,CPPAUTOBd27489741,2025-01-03 07:14:35,installation,installation,CPP,installation,MCAS Beaufort,
3657,FoodBeverage,dU9qwPsDEUpmuofHcWed56p2bOUL2VeY,SBD9515Q012,2025-01-02 02:20:52,Please share any additional feedback. Do not ...,Please share any additional feedback. Do not ...,CQ,OE_Feedback,Imrana opened the restaurant early and happily...,
3658,FoodBeverage,dU9qwPsDEUpmuofHcWed56p2bOUL2VeY,CPPAUTOBd27492941,2025-01-02 02:20:52,restaurantName,restaurantName,CPP,restaurantName,Quigley's Mainside,


In [8]:
df_customer_survey_responses_filtered[df_customer_survey_responses_filtered["questionId"].str.startswith("CPP", na=False)].count()

sheetName               2361
respondentId            2361
questionId              2361
responseTime            2361
questionName            2361
questionPhrase          2361
questionType            2361
questionLabel           2361
answerFreeTextValues    2361
sentiment               2361
dtype: int64

In [9]:
df_customer_survey_responses_filtered = df_customer_survey_responses_filtered[~df_customer_survey_responses_filtered["questionId"].str.startswith("CPP", na=False)]
print(f"Columns Left: {set(df_customer_survey_responses_filtered.columns)}")
print(df_customer_survey_responses_filtered.info())
print(df_customer_survey_responses_filtered["answerFreeTextValues"].notna().value_counts())
df_customer_survey_responses_filtered

Columns Left: {'questionId', 'sheetName', 'questionPhrase', 'answerFreeTextValues', 'responseTime', 'questionLabel', 'respondentId', 'sentiment', 'questionType', 'questionName'}
<class 'pandas.core.frame.DataFrame'>
Index: 1299 entries, 0 to 3657
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   sheetName             1299 non-null   object        
 1   respondentId          1299 non-null   object        
 2   questionId            1299 non-null   object        
 3   responseTime          1299 non-null   datetime64[ns]
 4   questionName          1299 non-null   object        
 5   questionPhrase        1299 non-null   object        
 6   questionType          1299 non-null   object        
 7   questionLabel         1299 non-null   object        
 8   answerFreeTextValues  1299 non-null   object        
 9   sentiment             1299 non-null   object        
dtypes: datetime64[ns](1

,sheetName,respondentId,questionId,responseTime,questionName,questionPhrase,questionType,questionLabel,answerFreeTextValues,sentiment
0,MainStores,NoEU4w4V910B5cBUo8VolA4C,SBD9516Q037,2025-02-17 15:24:00,What else would you like to share with us to h...,What else would you like to share with us to h...,CQ,OE_Improve,I was double charged for an item even said I t...,
1,MainStores,NoEU4w4V910B5cBUo8VolA4C,SBD9516Q039,2025-02-17 15:24:00,Please provide a brief description of your iss...,Please provide a brief description of your iss...,CQ,OE_Issue Description,I would like to be refunded the $10.99 plus ta...,
6,MainStores,MZNl01gAwZoMxUcE1sBRlA4C,SBD9516Q037,2025-02-17 10:39:38,What else would you like to share with us to h...,What else would you like to share with us to h...,CQ,OE_Improve,Please add video capture cards,
8,MainStores,RUIh8Jl9YldI95VYccMQoQ4C,SBD9516Q023,2025-02-17 10:30:40,Please specify your other primary reason for y...,Please specify your other primary reason for y...,CQ,MCX_Primary Reason Other,Running shoes,
10,MainStores,RJIZR9McwpBxcA9UxI1Idw4C,SBD9516Q037,2025-02-17 10:26:55,What else would you like to share with us to h...,What else would you like to share with us to h...,CQ,OE_Improve,Lower prices & better advertise your price mat...,
...,...,...,...,...,...,...,...,...,...,...
3639,FoodBeverage,dLVwWdg0wsdOmsTaZvtJLlB1YLwEvLNT,SBD9515Q012,2025-01-05 16:32:05,Please share any additional feedback. Do not ...,Please share any additional feedback. Do not ...,CQ,OE_Feedback,The Staff are exceptional,
3642,FoodBeverage,duXAVGoRiaFStFb7yv1gZLLznoLRZi7j,SBD9515Q012,2025-01-05 16:26:09,Please share any additional feedback. Do not ...,Please share any additional feedback. Do not ...,CQ,OE_Feedback,Sierra and Bell were outstanding. Both went ab...,
3647,FoodBeverage,QP8e2ufY5IzwhSqk28HmQeIFzQRxCrma,SBD9515Q012,2025-01-04 12:30:16,Please share any additional feedback. Do not ...,Please share any additional feedback. Do not ...,CQ,OE_Feedback,The staff is always amazing. Great management ...,
3654,FoodBeverage,GCCff7oOUUXiauSq4blkfl4a9EEuhfS7,SBD9515Q012,2025-01-03 07:14:35,Please share any additional feedback. Do not ...,Please share any additional feedback. Do not ...,CQ,OE_Feedback,The Epic burger was removed from the menu. Als...,


# Sentiment Analysis Labeling (Manual)

In [10]:
df_customer_survey_responses_filtered.to_csv(construct_path_from_project_root("../data-lake/MCCS_CustomerSurveyResponses_Original.csv"), index=False)
df_customer_survey_responses_filtered.to_excel(construct_path_from_project_root("../data-lake/MCCS_CustomerSurveyResponses_Original.xlsx"), index=False)

# Machine Learning Model Training and Evaluation

In [11]:
df_annotated = pd.read_csv(resolve_import_path_from_project_root("../data-lake/MCCS_CustomerSurveyResponses_Annotated.csv"))
df_annotated

,sheetName,respondentId,questionId,responseTime,questionName,questionPhrase,questionType,questionLabel,answerFreeTextValues,sentiment,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,MainStores,NoEU4w4V910B5cBUo8VolA4C,SBD9516Q037,2025-02-17 15:24:00,What else would you like to share with us to h...,What else would you like to share with us to h...,CQ,OE_Improve,I was double charged for an item even said I t...,N,NaN,NaN,NaN,NaN,NaN
1,MainStores,NoEU4w4V910B5cBUo8VolA4C,SBD9516Q039,2025-02-17 15:24:00,Please provide a brief description of your iss...,Please provide a brief description of your iss...,CQ,OE_Issue Description,I would like to be refunded the $10.99 plus ta...,N,NaN,NaN,NaN,NaN,NaN
2,MainStores,MZNl01gAwZoMxUcE1sBRlA4C,SBD9516Q037,2025-02-17 10:39:38,What else would you like to share with us to h...,What else would you like to share with us to h...,CQ,OE_Improve,Please add video capture cards,N,NaN,NaN,NaN,NaN,NaN
3,MainStores,RUIh8Jl9YldI95VYccMQoQ4C,SBD9516Q023,2025-02-17 10:30:40,Please specify your other primary reason for y...,Please specify your other primary reason for y...,CQ,MCX_Primary Reason Other,Running shoes,NaN,NaN,NaN,NaN,NaN,NaN
4,MainStores,RJIZR9McwpBxcA9UxI1Idw4C,SBD9516Q037,2025-02-17 10:26:55,What else would you like to share with us to h...,What else would you like to share with us to h...,CQ,OE_Improve,Lower prices & better advertise your price mat...,N,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1294,FoodBeverage,dLVwWdg0wsdOmsTaZvtJLlB1YLwEvLNT,SBD9515Q012,2025-01-05 16:32:05,Please share any additional feedback. Do not ...,Please share any additional feedback. Do not ...,CQ,OE_Feedback,The Staff are exceptional,G,NaN,NaN,NaN,NaN,NaN
1295,FoodBeverage,duXAVGoRiaFStFb7yv1gZLLznoLRZi7j,SBD9515Q012,2025-01-05 16:26:09,Please share any additional feedback. Do not ...,Please share any additional feedback. Do not ...,CQ,OE_Feedback,Sierra and Bell were outstanding. Both went ab...,G,NaN,NaN,NaN,NaN,NaN
1296,FoodBeverage,QP8e2ufY5IzwhSqk28HmQeIFzQRxCrma,SBD9515Q012,2025-01-04 12:30:16,Please share any additional feedback. Do not ...,Please share any additional feedback. Do not ...,CQ,OE_Feedback,The staff is always amazing. Great management ...,G,NaN,NaN,NaN,NaN,NaN
1297,FoodBeverage,GCCff7oOUUXiauSq4blkfl4a9EEuhfS7,SBD9515Q012,2025-01-03 07:14:35,Please share any additional feedback. Do not ...,Please share any additional feedback. Do not ...,CQ,OE_Feedback,The Epic burger was removed from the menu. Als...,N,NaN,NaN,NaN,NaN,NaN


In [12]:
df_annotated.drop(labels=df_annotated.columns[df_annotated.columns.str.startswith("Unnamed")], axis=1, inplace=True)
df_annotated

,sheetName,respondentId,questionId,responseTime,questionName,questionPhrase,questionType,questionLabel,answerFreeTextValues,sentiment
0,MainStores,NoEU4w4V910B5cBUo8VolA4C,SBD9516Q037,2025-02-17 15:24:00,What else would you like to share with us to h...,What else would you like to share with us to h...,CQ,OE_Improve,I was double charged for an item even said I t...,N
1,MainStores,NoEU4w4V910B5cBUo8VolA4C,SBD9516Q039,2025-02-17 15:24:00,Please provide a brief description of your iss...,Please provide a brief description of your iss...,CQ,OE_Issue Description,I would like to be refunded the $10.99 plus ta...,N
2,MainStores,MZNl01gAwZoMxUcE1sBRlA4C,SBD9516Q037,2025-02-17 10:39:38,What else would you like to share with us to h...,What else would you like to share with us to h...,CQ,OE_Improve,Please add video capture cards,N
3,MainStores,RUIh8Jl9YldI95VYccMQoQ4C,SBD9516Q023,2025-02-17 10:30:40,Please specify your other primary reason for y...,Please specify your other primary reason for y...,CQ,MCX_Primary Reason Other,Running shoes,NaN
4,MainStores,RJIZR9McwpBxcA9UxI1Idw4C,SBD9516Q037,2025-02-17 10:26:55,What else would you like to share with us to h...,What else would you like to share with us to h...,CQ,OE_Improve,Lower prices & better advertise your price mat...,N
...,...,...,...,...,...,...,...,...,...,...
1294,FoodBeverage,dLVwWdg0wsdOmsTaZvtJLlB1YLwEvLNT,SBD9515Q012,2025-01-05 16:32:05,Please share any additional feedback. Do not ...,Please share any additional feedback. Do not ...,CQ,OE_Feedback,The Staff are exceptional,G
1295,FoodBeverage,duXAVGoRiaFStFb7yv1gZLLznoLRZi7j,SBD9515Q012,2025-01-05 16:26:09,Please share any additional feedback. Do not ...,Please share any additional feedback. Do not ...,CQ,OE_Feedback,Sierra and Bell were outstanding. Both went ab...,G
1296,FoodBeverage,QP8e2ufY5IzwhSqk28HmQeIFzQRxCrma,SBD9515Q012,2025-01-04 12:30:16,Please share any additional feedback. Do not ...,Please share any additional feedback. Do not ...,CQ,OE_Feedback,The staff is always amazing. Great management ...,G
1297,FoodBeverage,GCCff7oOUUXiauSq4blkfl4a9EEuhfS7,SBD9515Q012,2025-01-03 07:14:35,Please share any additional feedback. Do not ...,Please share any additional feedback. Do not ...,CQ,OE_Feedback,The Epic burger was removed from the menu. Als...,N


In [13]:
print(df_annotated.info())
df_annotated["sentiment"].notna().value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1299 entries, 0 to 1298
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   sheetName             1299 non-null   object
 1   respondentId          1299 non-null   object
 2   questionId            1299 non-null   object
 3   responseTime          1299 non-null   object
 4   questionName          1299 non-null   object
 5   questionPhrase        1299 non-null   object
 6   questionType          1299 non-null   object
 7   questionLabel         1299 non-null   object
 8   answerFreeTextValues  1299 non-null   object
 9   sentiment             1260 non-null   object
dtypes: object(10)
memory usage: 101.6+ KB
None


sentiment
True     1260
False      39
Name: count, dtype: int64

In [14]:
df_annotated.dropna(subset=["sentiment"], inplace=True)
display(df_annotated)
print(df_annotated.info())
df_annotated["sentiment"].notna().value_counts()

,sheetName,respondentId,questionId,responseTime,questionName,questionPhrase,questionType,questionLabel,answerFreeTextValues,sentiment
0,MainStores,NoEU4w4V910B5cBUo8VolA4C,SBD9516Q037,2025-02-17 15:24:00,What else would you like to share with us to h...,What else would you like to share with us to h...,CQ,OE_Improve,I was double charged for an item even said I t...,N
1,MainStores,NoEU4w4V910B5cBUo8VolA4C,SBD9516Q039,2025-02-17 15:24:00,Please provide a brief description of your iss...,Please provide a brief description of your iss...,CQ,OE_Issue Description,I would like to be refunded the $10.99 plus ta...,N
2,MainStores,MZNl01gAwZoMxUcE1sBRlA4C,SBD9516Q037,2025-02-17 10:39:38,What else would you like to share with us to h...,What else would you like to share with us to h...,CQ,OE_Improve,Please add video capture cards,N
4,MainStores,RJIZR9McwpBxcA9UxI1Idw4C,SBD9516Q037,2025-02-17 10:26:55,What else would you like to share with us to h...,What else would you like to share with us to h...,CQ,OE_Improve,Lower prices & better advertise your price mat...,N
5,MainStores,5FxFJMks4h0VdEY8V9ZtEw4C,SBD9516Q037,2025-02-17 07:29:04,What else would you like to share with us to h...,What else would you like to share with us to h...,CQ,OE_Improve,Thanks,G
...,...,...,...,...,...,...,...,...,...,...
1294,FoodBeverage,dLVwWdg0wsdOmsTaZvtJLlB1YLwEvLNT,SBD9515Q012,2025-01-05 16:32:05,Please share any additional feedback. Do not ...,Please share any additional feedback. Do not ...,CQ,OE_Feedback,The Staff are exceptional,G
1295,FoodBeverage,duXAVGoRiaFStFb7yv1gZLLznoLRZi7j,SBD9515Q012,2025-01-05 16:26:09,Please share any additional feedback. Do not ...,Please share any additional feedback. Do not ...,CQ,OE_Feedback,Sierra and Bell were outstanding. Both went ab...,G
1296,FoodBeverage,QP8e2ufY5IzwhSqk28HmQeIFzQRxCrma,SBD9515Q012,2025-01-04 12:30:16,Please share any additional feedback. Do not ...,Please share any additional feedback. Do not ...,CQ,OE_Feedback,The staff is always amazing. Great management ...,G
1297,FoodBeverage,GCCff7oOUUXiauSq4blkfl4a9EEuhfS7,SBD9515Q012,2025-01-03 07:14:35,Please share any additional feedback. Do not ...,Please share any additional feedback. Do not ...,CQ,OE_Feedback,The Epic burger was removed from the menu. Als...,N


<class 'pandas.core.frame.DataFrame'>
Index: 1260 entries, 0 to 1298
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   sheetName             1260 non-null   object
 1   respondentId          1260 non-null   object
 2   questionId            1260 non-null   object
 3   responseTime          1260 non-null   object
 4   questionName          1260 non-null   object
 5   questionPhrase        1260 non-null   object
 6   questionType          1260 non-null   object
 7   questionLabel         1260 non-null   object
 8   answerFreeTextValues  1260 non-null   object
 9   sentiment             1260 non-null   object
dtypes: object(10)
memory usage: 108.3+ KB
None


sentiment
True    1260
Name: count, dtype: int64

In [15]:
df_annotated["sentiment"].value_counts()

sentiment
G    603
B    458
N    199
Name: count, dtype: int64

In [16]:
'''
Sources:
- LabelEncoder: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
'''
# Transforming "y" into a single-dimensional numerical vector
LABEL_ENCODER = LabelEncoder()
y_encoded = LABEL_ENCODER.fit_transform(df_annotated["sentiment"])
y_encoded

array([2, 2, 2, ..., 1, 2, 1])

In [17]:
label_encoder_mapping: Dict[int, str] = {index: label for index, label in enumerate(LABEL_ENCODER.classes_)}
print(f"Label Encoder Mapping: {label_encoder_mapping}")

Label Encoder Mapping: {0: 'B', 1: 'G', 2: 'N'}


In [18]:
with open(construct_path_from_project_root(f"src/models/ml_sentiment_analysis_label_encoder_mapping_results_{date_timestamp}.pkl"), "wb") as label_encoder_file:
    pickle.dump(obj=label_encoder_mapping, file=label_encoder_file)

In [19]:
# Setting up K-Fold configurations and instantiating the K-Fold object
k_fold = StratifiedKFold(n_splits=CONSTANTS.NUM_K_FOLDS, shuffle=True, random_state=CONSTANTS.RANDOM_STATE)
k_fold

StratifiedKFold(n_splits=10, random_state=5934, shuffle=True)

In [20]:
# Storing the Stratified K-Fold X_train, X_test, y_train, and y_test train-test split columns into an external Python dictionary data structure for later usage
k_fold_train_test_split_dict: Dict[str, pd.Series] = {}
for i, (train, test) in enumerate(k_fold.split(X=pd.concat([df_annotated["sheetName"], df_annotated["questionName"], df_annotated["questionType"], df_annotated["questionLabel"], df_annotated["answerFreeTextValues"]], axis=1), y=df_annotated["sentiment"])):
    k_fold_train_test_split_dict[f"X_train_SheetName_{i + 1}"] = df_annotated["sheetName"].iloc[train]
    k_fold_train_test_split_dict[f"X_train_QuestionName_{i + 1}"] = df_annotated["questionName"].iloc[train]
    k_fold_train_test_split_dict[f"X_train_QuestionType_{i + 1}"] = df_annotated["questionType"].iloc[train]
    k_fold_train_test_split_dict[f"X_train_QuestionLabel_{i + 1}"] = df_annotated["questionLabel"].iloc[train]
    k_fold_train_test_split_dict[f"X_train_AnswerFreeTextValues_{i + 1}"] = df_annotated["answerFreeTextValues"].iloc[train]
    
    k_fold_train_test_split_dict[f"X_test_SheetName_{i + 1}"] = df_annotated["sheetName"].iloc[test]
    k_fold_train_test_split_dict[f"X_test_QuestionName_{i + 1}"] = df_annotated["questionName"].iloc[test]
    k_fold_train_test_split_dict[f"X_test_QuestionType_{i + 1}"] = df_annotated["questionType"].iloc[test]
    k_fold_train_test_split_dict[f"X_test_QuestionLabel_{i + 1}"] = df_annotated["questionLabel"].iloc[test]
    k_fold_train_test_split_dict[f"X_test_AnswerFreeTextValues_{i + 1}"] = df_annotated["answerFreeTextValues"].iloc[test]
    
    k_fold_train_test_split_dict[f"y_train_{i + 1}"] = y_encoded[train]
    k_fold_train_test_split_dict[f"y_test_{i + 1}"] = y_encoded[test]
k_fold_train_test_split_dict

{'X_train_SheetName_1': 1         MainStores
 2         MainStores
 4         MainStores
 5         MainStores
 6         MainStores
             ...     
 1294    FoodBeverage
 1295    FoodBeverage
 1296    FoodBeverage
 1297    FoodBeverage
 1298    FoodBeverage
 Name: sheetName, Length: 1134, dtype: object,
 'X_train_QuestionName_1': 1       Please provide a brief description of your iss...
 2       What else would you like to share with us to h...
 4       What else would you like to share with us to h...
 5       What else would you like to share with us to h...
 6       What else would you like to share with us to h...
                               ...                        
 1294    Please share any additional feedback.  Do not ...
 1295    Please share any additional feedback.  Do not ...
 1296    Please share any additional feedback.  Do not ...
 1297    Please share any additional feedback.  Do not ...
 1298    Please share any additional feedback.  Do not ...
 Name: questi

In [21]:
# Loading in Spacy's USE model
SPACY_USE_NLP_MODEL = spacy_universal_sentence_encoder.load_model("en_use_md")
display(SPACY_USE_NLP_MODEL)

Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB



In [22]:
RF_CLASSIFIER = RandomForestClassifier(n_estimators=100, random_state=CONSTANTS.RANDOM_STATE)
display(RF_CLASSIFIER)

DT_CLASSIFIER = DecisionTreeClassifier(random_state=CONSTANTS.RANDOM_STATE)
display(DT_CLASSIFIER)

LOGISTIC_REGRESSOR = LogisticRegression(solver="lbfgs", multi_class="multinomial", random_state=CONSTANTS.RANDOM_STATE)
display(LOGISTIC_REGRESSOR)

ML_MODELS: List[Dict[str, Union[RandomForestClassifier, DecisionTreeClassifier, LogisticRegression]]] = [
    { RF_CLASSIFIER.__class__.__name__: RF_CLASSIFIER }, 
    { DT_CLASSIFIER.__class__.__name__: DT_CLASSIFIER }, 
    { LOGISTIC_REGRESSOR.__class__.__name__: LOGISTIC_REGRESSOR }
]
print(ML_MODELS)

RandomForestClassifier(random_state=5934)

DecisionTreeClassifier(random_state=5934)

LogisticRegression(multi_class='multinomial', random_state=5934)

[{'RandomForestClassifier': RandomForestClassifier(random_state=5934)}, {'DecisionTreeClassifier': DecisionTreeClassifier(random_state=5934)}, {'LogisticRegression': LogisticRegression(multi_class='multinomial', random_state=5934)}]


In [23]:
def compute_nlp_sentiment_analysis_classifier_performance(machine_learning_model: Union[RandomForestClassifier, \
                                        DecisionTreeClassifier, LogisticRegression], \
                                    machine_learning_model_name: str) \
                                    -> Tuple[float, float, float, float]:
    ml_model_accuracy_scores: List[float] = []
    ml_model_precision_scores: List[float] = []
    ml_model_recall_scores: List[float] = []
    ml_model_f1_scores: List[float] = []
    
    print(f"Using \"{machine_learning_model_name}\" for Performance Evaluation:")
    display(machine_learning_model)
    
    for i in CONSTANTS.RANGE_ITERATION_CROSS_VALIDATE:
        print(f"Iteration #{i}", "-" * 100)
        
        X_train_SheetName_series: pd.Series = k_fold_train_test_split_dict[f"X_train_SheetName_{i}"]
        sheetname_train_docs: List[Language] = []
        for (idx, item) in X_train_SheetName_series.items():
            sheetname_train_docs.append(SPACY_USE_NLP_MODEL(item))
        X_train_SheetName_use: List[np.ndarray] = list(map(lambda doc: doc.vector, sheetname_train_docs))
        
        X_train_QuestionName_series: pd.Series = k_fold_train_test_split_dict[f"X_train_QuestionName_{i}"]
        questionname_train_docs: List[Language] = []
        for (idx, item) in X_train_QuestionName_series.items():
            questionname_train_docs.append(SPACY_USE_NLP_MODEL(item))
        X_train_QuestionName_use: List[np.ndarray] = list(map(lambda doc: doc.vector, questionname_train_docs))
        
        X_train_QuestionType_series: pd.Series = k_fold_train_test_split_dict[f"X_train_QuestionType_{i}"]
        questiontype_train_docs: List[Language] = []
        for (idx, item) in X_train_QuestionType_series.items():
            questiontype_train_docs.append(SPACY_USE_NLP_MODEL(item))
        X_train_QuestionType_use: List[np.ndarray] = list(map(lambda doc: doc.vector, questiontype_train_docs))
        
        X_train_QuestionLabel_series: pd.Series = k_fold_train_test_split_dict[f"X_train_QuestionLabel_{i}"]
        questionlabel_train_docs: List[Language] = []
        for (idx, item) in X_train_QuestionLabel_series.items():
            questionlabel_train_docs.append(SPACY_USE_NLP_MODEL(item))
        X_train_QuestionLabel_use: List[np.ndarray] = list(map(lambda doc: doc.vector, questionlabel_train_docs))
        
        X_train_AnswerFreeTextValues_series: pd.Series = k_fold_train_test_split_dict[f"X_train_AnswerFreeTextValues_{i}"]
        answerfreetextvalues_train_docs: List[Language] = []
        for (idx, item) in X_train_AnswerFreeTextValues_series.items():
            answerfreetextvalues_train_docs.append(SPACY_USE_NLP_MODEL(item))
        X_train_AnswerFreeTextValues_use: List[np.ndarray] = list(map(lambda doc: doc.vector, answerfreetextvalues_train_docs))
        
        X_train: np.ndarray = np.hstack((X_train_SheetName_use, X_train_QuestionName_use, X_train_QuestionType_use, X_train_QuestionLabel_use, X_train_AnswerFreeTextValues_use))
        
        X_test_SheetName_series: pd.Series = k_fold_train_test_split_dict[f"X_test_SheetName_{i}"]
        sheetname_test_docs: List[Language] = []
        for (idx, item) in X_test_SheetName_series.items():
            sheetname_test_docs.append(SPACY_USE_NLP_MODEL(item))
        X_test_SheetName_use: List[np.ndarray] = list(map(lambda doc: doc.vector, sheetname_test_docs))
        
        X_test_QuestionName_series: pd.Series = k_fold_train_test_split_dict[f"X_test_QuestionName_{i}"]
        questionname_test_docs: List[Language] = []
        for (idx, item) in X_test_QuestionName_series.items():
            questionname_test_docs.append(SPACY_USE_NLP_MODEL(item))
        X_test_QuestionName_use: List[np.ndarray] = list(map(lambda doc: doc.vector, questionname_test_docs))
        
        X_test_QuestionType_series: pd.Series = k_fold_train_test_split_dict[f"X_test_QuestionType_{i}"]
        questiontype_test_docs: List[Language] = []
        for (idx, item) in X_test_QuestionType_series.items():
            questiontype_test_docs.append(SPACY_USE_NLP_MODEL(item))
        X_test_QuestionType_use: List[np.ndarray] = list(map(lambda doc: doc.vector, questiontype_test_docs))
        
        X_test_QuestionLabel_series: pd.Series = k_fold_train_test_split_dict[f"X_test_QuestionLabel_{i}"]
        questionlabel_test_docs: List[Language] = []
        for (idx, item) in X_test_QuestionLabel_series.items():
            questionlabel_test_docs.append(SPACY_USE_NLP_MODEL(item))
        X_test_QuestionLabel_use: List[np.ndarray] = list(map(lambda doc: doc.vector, questionlabel_test_docs))
        
        X_test_AnswerFreeTextValues_series: pd.Series = k_fold_train_test_split_dict[f"X_test_AnswerFreeTextValues_{i}"]
        answerfreetextvalues_test_docs: List[Language] = []
        for (idx, item) in X_test_AnswerFreeTextValues_series.items():
            answerfreetextvalues_test_docs.append(SPACY_USE_NLP_MODEL(item))
        X_test_AnswerFreeTextValues_use: List[np.ndarray] = list(map(lambda doc: doc.vector, answerfreetextvalues_test_docs))
        
        X_test: np.ndarray = np.hstack((X_test_SheetName_use, X_test_QuestionName_use, X_test_QuestionType_use, X_test_QuestionLabel_use, X_test_AnswerFreeTextValues_use))
        
        # Pass the X_train, y_train, and X_test vectors to the respective ML classifier
        y_train: pd.Series = k_fold_train_test_split_dict[f"y_train_{i}"]
        machine_learning_model.fit(X=X_train, y=y_train)
        y_pred = machine_learning_model.predict(X=X_test)
        
        # Compute and compare the predicted result (y_pred) against the source of truth (y_true) for an accuracy score
        y_true: pd.Series = k_fold_train_test_split_dict[f"y_test_{i}"]
        
        acc_score = round(float(accuracy_score(y_true, y_pred)), 6)
        ml_model_accuracy_scores.append(acc_score)
        
        pres_score = round(float(precision_score(y_true, y_pred, average="weighted")), 6)
        ml_model_precision_scores.append(pres_score)
        
        rec_score = round(float(recall_score(y_true, y_pred, average="weighted")), 6)
        ml_model_recall_scores.append(rec_score)
        
        f_score = round(float(f1_score(y_true, y_pred, average="weighted")), 6)
        ml_model_f1_scores.append(f_score)
        
        print(f"Current Metrics for Iteration {i}: {acc_score} (Accuracy), {pres_score} (Precision), {rec_score} (Recall), and {f_score} (F1)")
        
        print(f"Classification Report for Iteration {i}:")
        print("Note: Label Encoder Mapping -", label_encoder_mapping)
        print(classification_report(y_true, y_pred, target_names=sorted(label_encoder_mapping.values())))
        print()
    print("-" * 50)
    acc_score_mean = round(float(np.mean(ml_model_accuracy_scores)), 6)
    print(f"Accuracy Scores: {ml_model_accuracy_scores}, Mean: {acc_score_mean}")
    pres_score_mean = round(float(np.mean(ml_model_precision_scores)), 6)
    print(f"Precision Scores: {ml_model_precision_scores}, Mean: {pres_score_mean}")
    rec_score_mean = round(float(np.mean(ml_model_recall_scores)), 6)
    print(f"Recall Scores: {ml_model_recall_scores}, Mean: {rec_score_mean}")
    f1_score_mean = round(float(np.mean(ml_model_f1_scores)), 6)
    print(f"F1 Scores: {ml_model_f1_scores}, Mean: {f1_score_mean}")
    print("-" * 100)
    
    return acc_score_mean, pres_score_mean, rec_score_mean, f1_score_mean

In [24]:
max_f1_score = 0.0
best_classifier_name: Optional[str] = None
best_classifier: Optional[Union[RandomForestClassifier, DecisionTreeClassifier, LogisticRegression]] = None
for ml_model_dict in ML_MODELS:
    ml_model_name, ml_model = list(ml_model_dict.items())[0]
    _, _, _, f1_score_mean = compute_nlp_sentiment_analysis_classifier_performance(machine_learning_model_name=ml_model_name, machine_learning_model=ml_model)
    print(f"Average F1-Score for {ml_model_name}: {f1_score_mean}")
    # Optimizing for the highest F1 Score (balance between Precision and Recall - True Positives and False Positives/Negatives)
    if f1_score_mean > max_f1_score:
        max_f1_score = f1_score_mean
        best_classifier_name, best_classifier = ml_model_name, ml_model

Using "RandomForestClassifier" for Performance Evaluation:


RandomForestClassifier(random_state=5934)

Iteration #1 ----------------------------------------------------------------------------------------------------
Current Metrics for Iteration 1: 0.777778 (Accuracy), 0.784486 (Precision), 0.777778 (Recall), and 0.770055 (F1)
Classification Report for Iteration 1:
Note: Label Encoder Mapping - {0: 'B', 1: 'G', 2: 'N'}
              precision    recall  f1-score   support

           B       0.94      0.73      0.82        45
           G       0.75      0.93      0.83        61
           N       0.53      0.40      0.46        20

    accuracy                           0.78       126
   macro avg       0.74      0.69      0.70       126
weighted avg       0.78      0.78      0.77       126


Iteration #2 ----------------------------------------------------------------------------------------------------
Current Metrics for Iteration 2: 0.81746 (Accuracy), 0.824265 (Precision), 0.81746 (Recall), and 0.806918 (F1)
Classification Report for Iteration 2:
Note: Label Encoder Mapping - {0:

DecisionTreeClassifier(random_state=5934)

Iteration #1 ----------------------------------------------------------------------------------------------------
Current Metrics for Iteration 1: 0.714286 (Accuracy), 0.724528 (Precision), 0.714286 (Recall), and 0.718398 (F1)
Classification Report for Iteration 1:
Note: Label Encoder Mapping - {0: 'B', 1: 'G', 2: 'N'}
              precision    recall  f1-score   support

           B       0.79      0.73      0.76        45
           G       0.77      0.75      0.76        61
           N       0.46      0.55      0.50        20

    accuracy                           0.71       126
   macro avg       0.67      0.68      0.67       126
weighted avg       0.72      0.71      0.72       126


Iteration #2 ----------------------------------------------------------------------------------------------------
Current Metrics for Iteration 2: 0.690476 (Accuracy), 0.722219 (Precision), 0.690476 (Recall), and 0.703483 (F1)
Classification Report for Iteration 2:
Note: Label Encoder Mapping - {

...

/Users/andrewtran/Desktop/Andrew-Academics/Coding-Projects/CS-5934/CS-5934-Marines-LLM-Capstone-Project/python-prediction-model/py3_predict/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Current Metrics for Iteration 8: 0.865079 (Accuracy), 0.87504 (Precision), 0.865079 (Recall), and 0.866332 (F1)
Classification Report for Iteration 8:
Note: Label Encoder Mapping - {0: 'B', 1: 'G', 2: 'N'}
              precision    recall  f1-score   support

           B       0.92      0.78      0.85        46
           G       0.90      0.93      0.92        60
           N       0.68      0.85      0.76        20

    accuracy                           0.87       126
   macro avg       0.84      0.86      0.84       126
weighted avg       0.88      0.87      0.87       126


Iteration #9 ----------------------------------------------------------------------------------------------------


/Users/andrewtran/Desktop/Andrew-Academics/Coding-Projects/CS-5934/CS-5934-Marines-LLM-Capstone-Project/python-prediction-model/py3_predict/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Current Metrics for Iteration 9: 0.904762 (Accuracy), 0.906625 (Precision), 0.904762 (Recall), and 0.903463 (F1)
Classification Report for Iteration 9:
Note: Label Encoder Mapping - {0: 'B', 1: 'G', 2: 'N'}
              precision    recall  f1-score   support

           B       0.95      0.89      0.92        46
           G       0.88      0.97      0.92        60
           N       0.88      0.75      0.81        20

    accuracy                           0.90       126
   macro avg       0.90      0.87      0.88       126
weighted avg       0.91      0.90      0.90       126


Iteration #10 ----------------------------------------------------------------------------------------------------
Current Metrics for Iteration 10: 0.944444 (Accuracy), 0.944833 (Precision), 0.944444 (Recall), and 0.944069 (F1)
Classification Report for Iteration 10:
Note: Label Encoder Mapping - {0: 'B', 1: 'G', 2: 'N'}
              precision    recall  f1-score   support

           B       0.92      0.9

/Users/andrewtran/Desktop/Andrew-Academics/Coding-Projects/CS-5934/CS-5934-Marines-LLM-Capstone-Project/python-prediction-model/py3_predict/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [25]:
print(f"Best Classifier: \"{best_classifier_name}\", Best Average F1 Score: {max_f1_score}")
display(best_classifier)

Best Classifier: "LogisticRegression", Best Average F1 Score: 0.878259


LogisticRegression(multi_class='multinomial', random_state=5934)

In [26]:
with open(construct_path_from_project_root(f"src/models/ml_sentiment_analysis_model_{date_timestamp}.pkl"), "wb") as ml_model_file:
    pickle.dump(obj=best_classifier, file=ml_model_file)